In [ ]:
# fabric ssh setup
from fabric import Connection


classic_sender = Connection(
    host='pc538.emulab.net',
    user = 'PeterYao',
    port=22,
)




prague_sender = Connection(
    host='pc549.emulab.net',
    user = 'PeterYao',
    port=22,
)


dualq = Connection(
    host='pc557.emulab.net',
    user='PeterYao',
    port=22,
)

fifo_deep = Connection(
    host='pc524.emulab.net',
    user
    = 'PeterYao',  
    port=22,
)

rx = Connection(
    host='pc760.emulab.net',
    user = 'PeterYao',
    port=22,
)

classic_receiver = Connection(
    host='pc536.emulab.net',
    user = 'PeterYao',
    port = 25674,
)

prague_receiver = Connection(
    host='pc536.emulab.net',
    user =  'PeterYao',
    port= 25675,
)

connections = [classic_sender, classic_receiver, prague_sender, dualq, fifo_deep, rx, prague_receiver]

## Setup Queue Discipline for the first time. 

In [ ]:
# DualPi2
btl_capacity = 25 # Bottleneck capacity in Mbps
threshold = 1  # DUALPI2 target threshold in ms
bridge_iface = "br0"

# Commands to set up the bridge
cmds_bridge_setup = f'''
    sudo ip link add name {bridge_iface} type bridge
    sudo ip link set dev vlan179 master {bridge_iface}
    sudo ip link set dev eno3 master {bridge_iface}
    sudo ip link set dev {bridge_iface} up
'''

# Commands to set up queue discipline
cmds_prefix = f'''
    sudo tc qdisc del dev {bridge_iface} root
    sudo tc qdisc replace dev {bridge_iface} root handle 1: htb default 3
    sudo tc class add dev {bridge_iface} parent 1: classid 1:3 htb rate {btl_capacity}mbit
'''

cmds_specific = f'''
    sudo tc qdisc add dev {bridge_iface} parent 1:3 handle 3: dualpi2 target {threshold}ms
'''

# cmds_specific = f'''
#     sudo tc qdisc add dev {bridge_iface} parent 1:3 handle 3: fq limit 833 flow_limit 833 orphan_mask 0 ce_threshold {threshold}ms
# '''

# cmds_specific = f'''
#     sudo tc qdisc add dev {bridge_iface} parent 1:3 handle 3: fq_codel limit 209 target 1ms interval 100ms ecn
# '''


# Command to verify setup
cmds_show = f"sudo tc qdisc show dev {bridge_iface}"

# Execute commands on the dualq node
dualq.run(cmds_bridge_setup)
dualq.run(cmds_prefix)
dualq.run(cmds_specific)
dualq.run(cmds_show)


flush_and_addign_cmd = '''
sudo ip addr flush dev vlan179
sudo ip addr flush dev eno3

# Assign both IP addresses to the bridge using aliasing
sudo ip addr add 10.0.8.1/24 dev br0
sudo ip addr add 10.0.1.2/24 dev br0
'''

dualq.run(flush_and_addign_cmd)

# remember to use traceroute to verify the path

In [ ]:
# setup the deep fifo node

# merge 2 interfaces to apply a single queue discipline
# Variables
n_bdp = 2
base_rtt = 40  # in ms
btl_capacity = 25  # in Mbps

# Compute the buffer limit based on BDP
# btl_limit = int(1000 * n_bdp * btl_capacity * base_rtt / 8)  # in bytes
btl_limit = 0.625 * 1000 * 1000 / 2
packet_number = int(btl_limit / 1500) + 1

print("btl limit: ", btl_limit)
print("packet number: ", packet_number)
print("htb rate : ", btl_capacity)

# Interface and command preparation
bridge_iface = "br0"

cmds_bridge_setup = f'''
    sudo ip link add name {bridge_iface} type bridge
    sudo ip link set dev eno3 master {bridge_iface}
    sudo ip link set dev vlan165 master {bridge_iface}
    sudo ip link set dev vlan186 master {bridge_iface}
    sudo ip link set dev {bridge_iface} up
'''

# cmds_qdisc_setup = f'''
#     sudo tc qdisc del dev {bridge_iface} root
#     sudo tc qdisc replace dev {bridge_iface} root handle 1: tbf default 3
#     sudo tc class add dev {bridge_iface} parent 1: classid 1:3 tbf rate {btl_capacity}mbit
#     sudo tc qdisc add dev {bridge_iface} parent 1:3 handle 3: bfifo limit {btl_limit}
# '''

cmds_qdisc_setup = f'''
    sudo tc qdisc del dev {bridge_iface} root
    sudo tc qdisc add dev {bridge_iface} root tbf rate {btl_capacity}mbit limit {btl_limit} burst 200kbit
'''

cmds_show = f"sudo tc qdisc show dev {bridge_iface}"

# Run commands on fifo_deep
fifo_deep.run(cmds_bridge_setup)
fifo_deep.run(cmds_qdisc_setup)
fifo_deep.run(cmds_show)


flush_cmd = '''
sudo ip addr flush dev vlan165
sudo ip addr flush dev vlan186
sudo ip addr flush dev eno3
'''

fifo_deep.run(flush_cmd)

add_ip_cmd = '''
sudo ip addr add 10.0.6.1/24 dev br0
sudo ip addr add 10.0.0.2/24 dev br0
sudo ip addr add 10.0.7.1/24 dev br0
'''

fifo_deep.run(add_ip_cmd)

# feel free to double check, but the ip route should be already set up correctly.  

## Adjust DeepFIFO queue Disc After First Setup
for separate queue case, we use 0.3125mb and 25mbps egress rate, for shared q case, we use 0.625mb and 50mbps egress rate. 


### Separate Queue Case. 

In [ ]:
# setup the deep fifo node

# merge 2 interfaces to apply a single queue discipline
# Variables
n_bdp = 2
base_rtt = 40  # in ms
btl_capacity = 25  # in Mbps

# Compute the buffer limit based on BDP
# btl_limit = int(1000 * n_bdp * btl_capacity * base_rtt / 8)  # in bytes
btl_limit = 0.625 * 1000 * 1000 / 2
packet_number = int(btl_limit / 1500) + 1

print("btl limit: ", btl_limit)
print("packet number: ", packet_number)
print("htb rate : ", btl_capacity)

# Interface and command preparation
bridge_iface = "br0"

cmds_bridge_setup = f'''
    sudo ip link add name {bridge_iface} type bridge
    sudo ip link set dev eno3 master {bridge_iface}
    sudo ip link set dev vlan165 master {bridge_iface}
    sudo ip link set dev vlan186 master {bridge_iface}
    sudo ip link set dev {bridge_iface} up
'''

# cmds_qdisc_setup = f'''
#     sudo tc qdisc del dev {bridge_iface} root
#     sudo tc qdisc replace dev {bridge_iface} root handle 1: tbf default 3
#     sudo tc class add dev {bridge_iface} parent 1: classid 1:3 tbf rate {btl_capacity}mbit
#     sudo tc qdisc add dev {bridge_iface} parent 1:3 handle 3: bfifo limit {btl_limit}
# '''

cmds_qdisc_setup = f'''
    sudo tc qdisc del dev {bridge_iface} root
    sudo tc qdisc add dev {bridge_iface} root tbf rate {btl_capacity}mbit limit {btl_limit} burst 200kbit
'''

cmds_show = f"sudo tc qdisc show dev {bridge_iface}"

# Run commands on fifo_deep
# fifo_deep.run(cmds_bridge_setup)
fifo_deep.run(cmds_qdisc_setup)
fifo_deep.run(cmds_show)


flush_cmd = '''
sudo ip addr flush dev vlan165
sudo ip addr flush dev vlan186
sudo ip addr flush dev eno3
'''

# fifo_deep.run(flush_cmd)

add_ip_cmd = '''
sudo ip addr add 10.0.6.1/24 dev br0
sudo ip addr add 10.0.0.2/24 dev br0
sudo ip addr add 10.0.7.1/24 dev br0
'''

# fifo_deep.run(add_ip_cmd)

# feel free to double check, but the ip route should be already set up correctly. 


 

### Shared Queue Case. 

In [ ]:
# setup the deep fifo node

# merge 2 interfaces to apply a single queue discipline
# Variables
n_bdp = 2
base_rtt = 40  # in ms
btl_capacity = 50  # in Mbps

# Compute the buffer limit based on BDP
# btl_limit = int(1000 * n_bdp * btl_capacity * base_rtt / 8)  # in bytes
btl_limit = 0.625 * 1000 * 1000
packet_number = int(btl_limit / 1500) + 1

print("btl limit: ", btl_limit)
print("packet number: ", packet_number)
print("htb rate : ", btl_capacity)

# Interface and command preparation
bridge_iface = "br0"

cmds_bridge_setup = f'''
    sudo ip link add name {bridge_iface} type bridge
    sudo ip link set dev eno3 master {bridge_iface}
    sudo ip link set dev vlan165 master {bridge_iface}
    sudo ip link set dev vlan186 master {bridge_iface}
    sudo ip link set dev {bridge_iface} up
'''

# cmds_qdisc_setup = f'''
#     sudo tc qdisc del dev {bridge_iface} root
#     sudo tc qdisc replace dev {bridge_iface} root handle 1: tbf default 3
#     sudo tc class add dev {bridge_iface} parent 1: classid 1:3 tbf rate {btl_capacity}mbit
#     sudo tc qdisc add dev {bridge_iface} parent 1:3 handle 3: bfifo limit {btl_limit}
# '''

cmds_qdisc_setup = f'''
    sudo tc qdisc del dev {bridge_iface} root
    sudo tc qdisc add dev {bridge_iface} root tbf rate {btl_capacity}mbit limit {btl_limit} burst 200kbit
'''

cmds_show = f"sudo tc qdisc show dev {bridge_iface}"

# Run commands on fifo_deep
# fifo_deep.run(cmds_bridge_setup)
fifo_deep.run(cmds_qdisc_setup)
fifo_deep.run(cmds_show)


flush_cmd = '''
sudo ip addr flush dev vlan165
sudo ip addr flush dev vlan186
sudo ip addr flush dev eno3
'''

# fifo_deep.run(flush_cmd)

add_ip_cmd = '''
sudo ip addr add 10.0.6.1/24 dev br0
sudo ip addr add 10.0.0.2/24 dev br0
sudo ip addr add 10.0.7.1/24 dev br0
'''

# fifo_deep.run(add_ip_cmd)

# feel free to double check, but the ip route should be already set up correctly. 


 